In [34]:
# import for data handling
import pandas as pd
# import for numerical analysis
import numpy as np

# read the data from the csv file into a dataframe
titanic = pd.read_csv("./titanic.csv")
titanic.shape

(887, 9)

In [24]:
titanic[titanic.Survived == 1].groupby("Sex").Survived.count()

Sex
female    233
male      109
Name: Survived, dtype: int64

## Part 1
Fit an appropriate regression model to predict the survival of a passenger using sex as a single predictor. With the help of the fitted model, answer the following questions (show your calculations,
either by hand or with help of R):
• What is the probability that you survive given that you are a male?
• What is the probability of surviving if you are female?

In [11]:
# import modules for regression using formula
import statsmodels.formula.api as smf
import statsmodels.api as sm

# formula for calculating survival based on sex
formula = "Survived ~ C(Sex)"

# create and fit the model for logistic regression using glm with the formula
model1 = smf.glm(formula=formula, data=titanic, family=sm.families.Binomial()).fit()
# calculate and print the summary of the model created
print("Summary for model for calculating Survival based on sex of passenger :\n\n",model1.summary())

# use the model created to make predictions for both genders

# survival probability prediction for sex = male
probability_survival_male = model1.predict(pd.DataFrame({'Sex': ["male"]}))
print('\nSurvival probability percentage for males is: ', probability_survival_male.values[0]*100, "%")
# calculate and print the odds of survival for males using the formula: odds = p/(1-p)
odd_survival_male = probability_survival_male.values[0]/(1-probability_survival_male.values[0])
print('Survival odds for males are: ', odd_survival_male, " to 1")

# survival probability prediction for sex = female
probability_survival_female = model1.predict(pd.DataFrame({'Sex': ["female"]}))
print('\nSurvival probability percentage for females is: ', probability_survival_female.values[0]*100, "%")
# calculate and print the odds of survival for females using the formula: odds = p/(1-p)
odd_survival_female = probability_survival_female.values[0]/(1-probability_survival_female.values[0])
print('Survival odds for females are: ', odd_survival_female, " to 1")

Summary for model for calculating Survival based on sex of passenger :

                  Generalized Linear Model Regression Results                  
Dep. Variable:               Survived   No. Observations:                  887
Model:                            GLM   Df Residuals:                      885
Model Family:                Binomial   Df Model:                            1
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -458.06
Date:                Tue, 09 Nov 2021   Deviance:                       916.12
Time:                        00:41:29   Pearson chi2:                     887.
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------

-----------------------------

## PART 2
Remember the character ’Jack Dawson’ from the movie Titanic? Using the dataset, fit appropriate regression models to answer whether his death in the movie is realistic or not. You are free
to choose the explanatory variables to be included in the model. The following is known about
Jack:
- male
- 20 years old
- travelled in the third class
- travelled without any family
- ticket fare unknown since he won it in a game of cards
- embarked in Southampton

You can compare several models, but make sure that you clearly state the final model chosen and
the reasons behind this final selection. With the help of your final model chosen, interpret the
results in terms of probability of survival (as you did in Part 1)

In [12]:
# we rename the columns where there are spaces in the name so that we can easily use them in the formula
titanic = titanic.rename(columns={"Siblings/Spouses Aboard": "Siblings_Spouses", "Parents/Children Aboard": "Parents_Children"})

In [29]:
# function to perform backward stepwise regression to find optimal features we should include in the formula
# it takes the features list as input
# it gives a dictionary of AIC score with the corresponding model formula used , as the output
def stepwise_regression(included):
    # initialize a dictionary for the output
    model_formula_and_aic = {}
    print("Performing backward stepwise regression to find optimal formula ---->")
    # run a loop to perform backward regression till model changes
    while True:
        # stooping variable to end loop of backward regression when model stops changes
        # initially set as False
        altered = False
        # evaluate the formula to be used for the model from the included features list
        formula_new = "Survived ~ " + " + ".join(included)
        # create and fit the glm model using the formula
        model = smf.glm(formula=formula_new, data=titanic, family=sm.families.Binomial()).fit()
        # calculate the AIC score for the model created
        aic = model.aic
        # print the formula and the AIC score for the model evaluated
        print("\nWith formula [", formula_new, "], \nAIC score of model = ", aic)
        # store the formula used and the AIC score for the model in the dictionary, with AIC as the key
        model_formula_and_aic[aic] = formula_new
        # separate all coefficients except the intercept
        pvalues = model.pvalues.iloc[1:]
        # calculate the maximum pvalue
        worst_pval = pvalues.max()
        
        # set pvalue threshold at 0.05, if max pvalue is greater than that, we remove the feature
        if worst_pval > 0.05:
            # if value is greater than threshold set loop stopping variable to True, to indicate change made in this iteration
            # when set to true, it ensures the loop carries on
            altered = True
            # get the name of the feature with the maximum pvalue (the worst feature)
            worst_feature = str(pvalues.idxmax())
            # if the feature name has a specific Reference group value appended to its name, we remove the appended value from the string
            if "[" in worst_feature:
                worst_feature = worst_feature[0: worst_feature.index("[")]
            # remove the worst feature from the list of included columns/features
            included.remove(worst_feature)
            # print the feature being dropped with its p value for logging
            print('\n - Dropping the feature = ', worst_feature ,', with p-value = ',  worst_pval)
       
        # if no change is done in this iteration that means stepwise regression process has ended, end the loop
        if not altered:
            break
   
    # return the dictionary containing all the models' AIC scores and Formulas used        
    return model_formula_and_aic

In [30]:
# create a list of all features that we have for JACK
included_columns = ["C(Sex)", "C(Embarked)", "Age", "C(Pclass)", "Siblings_Spouses", "Parents_Children", "Fare"]
# call the stepwise regression method for these features
model_formulas_aic = stepwise_regression(included_columns)

Performing backward stepwise regression to find optimal formula ---->

With formula [ Survived ~ C(Sex) + C(Embarked) + Age + C(Pclass) + Siblings_Spouses + Parents_Children + Fare ], 
AIC score of model =  799.7065180109817

 - Dropping the feature =  C(Embarked) , with p-value =  0.48293802692736554

With formula [ Survived ~ C(Sex) + Age + C(Pclass) + Siblings_Spouses + Parents_Children + Fare ], 
AIC score of model =  796.9281554818747

 - Dropping the feature =  Parents_Children , with p-value =  0.3681512920141502

With formula [ Survived ~ C(Sex) + Age + C(Pclass) + Siblings_Spouses + Fare ], 
AIC score of model =  795.7532687957257

 - Dropping the feature =  Fare , with p-value =  0.31965839309099664

With formula [ Survived ~ C(Sex) + Age + C(Pclass) + Siblings_Spouses ], 
AIC score of model =  794.8181416117563


In [31]:
# now we compare the AIC scores of these models to select the optimal model
# the model with the lowest AIC score is the best

# calculate the minimum AIC score from the dictionary of AIC scores and formulas, we got from the stepwise regression method
min_aic = min(model_formulas_aic.keys())
# get the best formula corresponding to the minimum AIC score and print it
best_formula = model_formulas_aic[min_aic]
print("The model with the best aic score has the formula = [", best_formula, "]")

The model with the best aic score has the formula = [ Survived ~ C(Sex) + Age + C(Pclass) + Siblings_Spouses ]


In [32]:
# create and fit a new model with the optimal formula calculated
model2 = smf.glm(formula=best_formula, data=titanic, family=sm.families.Binomial()).fit()
# print the summary for the model
print("Summary of selected model : \n\n ",model2.summary())

Summary of selected model : 

                   Generalized Linear Model Regression Results                  
Dep. Variable:               Survived   No. Observations:                  887
Model:                            GLM   Df Residuals:                      881
Model Family:                Binomial   Df Model:                            5
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -391.41
Date:                Tue, 09 Nov 2021   Deviance:                       782.82
Time:                        14:50:02   Pearson chi2:                     921.
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercep

In [33]:
# use the model to predict probability of Jack Dawson's survival and print it
probability_survival_jack = model2.predict(pd.DataFrame({"Sex": ["male"], "Age": [20], "Pclass": [3], "Siblings_Spouses": [0]}))
print('Survival probability percentage for Jack Dawson is: ', probability_survival_jack.values[0]*100, "%")
# calculate and print the odds of survival for jack using the formula: odds = p/(1-p)
odd_survival_jack = probability_survival_jack.values[0]/(1-probability_survival_jack.values[0])
print('Survival odds for Jack are: ', odd_survival_jack, " to 1")

Survival probability percentage for Jack Dawson is:  13.43001420868069 %
Survival odds for Jack are:  0.15513476276933116  to 1
